# Using the high-level API to verify the robustness of the Ensemble trees.

In [1]:
import sys
sys.path.append("..")

In [2]:
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
import xgboost
from sklearn.ensemble import RandomForestClassifier
from satreeverify import SATreeAttack

/home/masoud/anaconda3/lib/python3.8/site-packages/xgboost/compat.py:31: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [3]:
data = load_breast_cancer()
X = data['data']
y = data["target"]
feature_names = data["feature_names"]
y = y==1

In [4]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# XGBoost Model

In [5]:
ntrees = 50
clf = xgboost.XGBClassifier(n_estimators=ntrees, max_depth=3).fit(x_train, y_train)
print(clf.score(x_test, y_test))

attack = SATreeAttack(clf)

/home/masoud/anaconda3/lib/python3.8/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[14:23:00] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
0.9824561403508771


In [6]:
epsilon = 0.4

index = 105
sample = X[index:index+1, :]
clf.predict_proba(sample)

array([[0.9903275 , 0.00967252]], dtype=float32)

- x_adv is the SAT results
- x_adv_sample is the x_adv transformed to the original input space
- compare is a pandas dataframe that compares the original input and the x_adv_sample

In [7]:
result = attack.soft_attack(sample, epsilon)
result["comparison"]

,0,1,diff(%),bound
0,13.110000,13.110000,0.000000,"[-inf, 15.0450001]"
1,15.560000,16.562121,0.064404,"[16.5600014, 18.6800003]"
2,87.210000,87.210000,0.000000,"[-inf, inf]"
3,530.200000,530.200000,0.000000,"[-inf, 689.450012]"
4,0.139800,0.089955,0.356547,"[0.0896499977, 0.0899550021]"
5,0.176500,0.176500,0.000000,"[0.0716100037, inf]"
6,0.207100,0.207100,0.000000,"[0.0975949988, inf]"
7,0.096010,0.071987,0.250211,"[0.0492300019, 0.072010003]"
8,0.192500,0.192500,0.000000,"[0.166500002, inf]"
9,0.076920,0.076920,0.000000,"[-inf, inf]"


In [8]:
clf.predict(sample), clf.predict(result["adv_sample"])

(array([False]), array([ True]))

- x_adv is the SAT results
- x_adv_sample is the x_adv transformed to the original input space
- compare is a pandas dataframe that compares the original input and the x_adv_sample

In [9]:
result = attack.hard_attack(sample, epsilon, 8) # hard attack binarizes the decision values. here we use 8 bits to binarize the decision values
result["comparison"]

,0,1,diff(%),bound
0,13.110000,15.060045,0.148745,"[15.0450001, inf]"
1,15.560000,14.975010,0.037596,"[-inf, 14.9899998]"
2,87.210000,87.210000,0.000000,"[-inf, inf]"
3,530.200000,530.200000,0.000000,"[-inf, 696.25]"
4,0.139800,0.108632,0.222948,"[0.0904600024, 0.108649999]"
5,0.176500,0.176500,0.000000,"[0.0716100037, inf]"
6,0.207100,0.207100,0.000000,"[0.0975949988, inf]"
7,0.096010,0.071987,0.250211,"[0.0492300019, 0.072010003]"
8,0.192500,0.166334,0.135930,"[-inf, 0.166500002]"
9,0.076920,0.076920,0.000000,"[-inf, inf]"


In [10]:
clf.predict(sample), clf.predict(result["adv_sample"])

(array([False]), array([ True]))

# RandomForest Model

In [11]:
clf = RandomForestClassifier(n_estimators=ntrees, max_depth=3).fit(X, y)
print(clf.score(x_test, y_test))

attack = SATreeAttack(clf)

1.0


In [12]:
epsilon = 0.4

index = 105
sample = X[index:index+1, :]
clf.predict_proba(sample)

array([[0.84967368, 0.15032632]])

- x_adv is the SAT results
- x_adv_sample is the x_adv transformed to the original input space
- compare is a pandas dataframe that compares the original input and the x_adv_sample

In [13]:
result = attack.soft_attack(sample, epsilon)
result["comparison"]

,0,1,diff(%),bound
0,13.110000,10.302687,0.214135,"[-inf, 10.312999725341797]"
1,15.560000,14.954185,0.038934,"[14.139999866485596, 14.954999923706055]"
2,87.210000,72.513513,0.168518,"[71.0250015258789, 72.5150032043457]"
3,530.200000,530.200000,0.000000,"[-inf, 536.4000091552734]"
4,0.139800,0.108443,0.224301,"[0.10114999860525131, 0.10844999924302101]"
5,0.176500,0.134932,0.235514,"[0.11674999818205833, 0.13494999706745148]"
6,0.207100,0.137328,0.336900,"[0.11534999683499336, 0.13735000044107437]"
7,0.096010,0.096010,0.000000,"[0.06218999996781349, inf]"
8,0.192500,0.149995,0.220804,"[0.1453000009059906, 0.14999999850988388]"
9,0.076920,0.080060,0.040821,"[0.07997999712824821, inf]"


In [14]:
clf.predict(sample), clf.predict(result["adv_sample"])

(array([False]), array([ True]))

- x_adv is the SAT results
- x_adv_sample is the x_adv transformed to the original input space
- compare is a pandas dataframe that compares the original input and the x_adv_sample

In [15]:
result = attack.hard_attack(sample, epsilon, 8)
result["comparison"]

,0,1,diff(%),bound
0,13.110000,10.302687,0.214135,"[-inf, 10.312999725341797]"
1,15.560000,15.014975,0.035027,"[14.990000247955322, 15.015000343322754]"
2,87.210000,72.442488,0.169333,"[-inf, 72.5150032043457]"
3,530.200000,696.050218,0.312807,"[696.0500183105469, 696.25]"
4,0.139800,0.101130,0.276611,"[0.0809599980711937, 0.10114999860525131]"
5,0.176500,0.176500,0.000000,"[0.16804999858140945, 0.1906999945640564]"
6,0.207100,0.156059,0.246455,"[0.11534999683499336, 0.15610000491142273]"
7,0.096010,0.060396,0.370943,"[0.05615000054240227, 0.06039999984204769]"
8,0.192500,0.192500,0.000000,"[0.14999999850988388, inf]"
9,0.076920,0.076920,0.000000,"[0.061650000512599945, 0.07997999712824821]"


In [16]:
clf.predict(sample), clf.predict(result["adv_sample"])

(array([False]), array([ True]))